# **cookbook** | webscraping

Data extraction from webpage: https://uwielbiam.pl

In [1]:
#! pip install httplib2

In [2]:
import re
import numpy as np
import pandas as pd
import httplib2
import requests
from bs4 import BeautifulSoup, SoupStrainer
from collections import Counter
from tqdm import tqdm

import warnings
warnings.filterwarnings("ignore")

### MAIN DISHES

In [3]:
class Webpage(object):
    
    def __init__(self, link):
        self.link = link
        self.link_przepisy = link + '/przepisy?pg='
        self.link_przepis = link + '/przepis/'
        
    def one_page_meals(self, number):
        """Return list with all dishes from a particular page"""
        if number > 0:
            all_links = []
            http = httplib2.Http()
            status, response = http.request(self.link_przepisy + str(number))
            for link in BeautifulSoup(response, 'html.parser', parseOnlyThese=SoupStrainer('a')):
                if link.has_attr('href') and link['href'].startswith('/przepis/'):
                    all_links.append(((link['href'])[len('/przepis/'):]).replace('-', ' '))
            return all_links
        else:
            print("number variable needs to be greater than zero.")

    def one_page_links(self, number):
        """Return list with all dishes links from a particular page"""
        if number > 0:
            all_links = []
            http = httplib2.Http()
            status, response = http.request(self.link_przepisy + str(number))
            for link in BeautifulSoup(response, 'html.parser', parseOnlyThese=SoupStrainer('a')):
                if link.has_attr('href') and link['href'].startswith('/przepis/'):
                    all_links.append(self.link_przepis + (link['href'])[len('/przepis/'):])
            return all_links
        else:
            print("number variable needs to be greater than zero.")
            
    @property
    def all_links(self):
        """Return list with all dishes names from the webpage"""
        all_pages = 458
        if number > 0:
            all_links = []
            for i in range(1, all_pages):
                if i%10 == 0:
                    print(f'{i}/{all_pages}', end='\r')
                http = httplib2.Http()
                status, response = http.request(self.link_przepisy + str(i))
                for link in BeautifulSoup(response, 'html.parser', parseOnlyThese=SoupStrainer('a')):
                    if link.has_attr('href') and link['href'].startswith('/przepis/'):
                        all_links.append(self.link_przepis + (link['href'])[len('/przepis/'):])
            return all_links
        else:
            print("number variable needs to be greater than zero.")

In [4]:
uwielbiam = Webpage('http://uwielbiam.pl/')
uwielbiam.link

'http://uwielbiam.pl/'

In [5]:
uwielbiam.one_page_meals(1)

['salatka z brukselka i ogorkami kiszonymi krakus',
 'salatka z gruszka i serem plesniowym',
 'salatka z winogronem i serem plesniowym',
 'salatka z figami i szynka parmenska',
 'salatka z winogronem i kurczakiem',
 'salatka ziemniaczana z bobem boczkiem i korniszonami z chili i francuskim ',
 'salatka z pieczonych burakow i mozzarelli z dressingiem z syropu malinowego 2',
 'mini makaron czekoladowy z truskawkami arbuzem i sosem czekoladowym',
 'salatka z makaronem czekoladowym feta arbuzem zielonymi oliwkami i cebula',
 'wiosenna salatka z kalarepy z ogorkami konserwowymi krakus',
 'salatka ziemniaczana z ogorkami kiszonymi krakus',
 'salatka gyros ale grecka z ogorkami korniszony z chili',
 'salatka makaronowa z pieczonym miesem',
 'makaron ryzowy z awokado jajkiem mozzarella i sosem bazyliowym',
 'salatka jajeczna z paluszkami krabowymi',
 'salatka z jarmuzu i awokado',
 'kasza peczak ze szpinakiem',
 'salatka z awokado i lososiem',
 'salatka z granatem i kurczakiem',
 'salatka jarz

In [6]:
uwielbiam.one_page_links(1)

['http://uwielbiam.pl//przepis/salatka-z-brukselka-i-ogorkami-kiszonymi-krakus',
 'http://uwielbiam.pl//przepis/salatka-z-gruszka-i-serem-plesniowym',
 'http://uwielbiam.pl//przepis/salatka-z-winogronem-i-serem-plesniowym',
 'http://uwielbiam.pl//przepis/salatka-z-figami-i-szynka-parmenska',
 'http://uwielbiam.pl//przepis/salatka-z-winogronem-i-kurczakiem',
 'http://uwielbiam.pl//przepis/salatka-ziemniaczana-z-bobem-boczkiem-i-korniszonami-z-chili-i-francuskim-',
 'http://uwielbiam.pl//przepis/salatka-z-pieczonych-burakow-i-mozzarelli-z-dressingiem-z-syropu-malinowego-2',
 'http://uwielbiam.pl//przepis/mini-makaron-czekoladowy-z-truskawkami-arbuzem-i-sosem-czekoladowym',
 'http://uwielbiam.pl//przepis/salatka-z-makaronem-czekoladowym-feta-arbuzem-zielonymi-oliwkami-i-cebula',
 'http://uwielbiam.pl//przepis/wiosenna-salatka-z-kalarepy-z-ogorkami-konserwowymi-krakus',
 'http://uwielbiam.pl//przepis/salatka-ziemniaczana-z-ogorkami-kiszonymi-krakus',
 'http://uwielbiam.pl//przepis/salatka-

### DISH PROPERTIES

In [7]:
class Uwielbiam_dish_page(object):
    
    def __init__(self, link):
        self.link = link
        self.html_content = BeautifulSoup(requests.get(link).content)
        
    @property
    def title(self):
        title_content = self.html_content.find('h1', attrs = {'class': 'h-section'})
        return title_content.text
    
    @property
    def description(self):
        description_content = self.html_content.find('p', attrs = {'class': 'p--description'})
        try:
            description_content = description_content.text.replace('\u2028\r', ' ').replace('  \n', ' ').replace('\r\n', '')
        except:
            description_content = 'None'
        return description_content
    
    @property
    def ingredients(self):
        ingr = self.html_content.find_all('tr', attrs = {'class': 'ingredients_row'})
        x_list = []
        for x in ingr:
            x_list.append(x.text.split())
        ingredients = []
        for x in x_list[:-2]:
            try: 
                x1 = int(x[0])
                x2 = x[1]
                x3 = ' '.join(x[2:])
                ingredients.append([x1, x2, x3])
            except:
                ingredients.append([' '.join(x)])
            pass
        return ingredients
    
    @property
    def preparation(self):
        preparation_content = self.html_content.find_all('li', attrs = {'class': 'recipe-list--row'})
        prep_content = []
        for i in range(len(preparation_content)):
            prep_content.append(preparation_content[i].text.strip().replace('\r\n', ''))
        return prep_content
    
    @property
    def tags(self):
        tags = self.html_content.find_all('a', attrs = {'class': 'tag f-element'})
        tag_content = []
        for n in range(len(tags)):
            tag_content.append(tags[n].text.lower())
        return tag_content
    
    @property
    def allergens(self):
        ingr = self.html_content.find_all('tr', attrs = {'class': 'ingredients_row'})
        x_list = []
        for x in ingr:
            x_list.append(x.text.split())
        save = 0
        allergens = []
        for counter, item in enumerate(x_list[-2]):
            if item == "Alergeny":
                save = 1
            else:
                if save == 1:
                    allergens.append(item)
                else:
                    pass
        if allergens == []:
            return 'None'
        return allergens
    
    @property
    def skill(self):
        return self.html_content.find_all('span', attrs = {'class': 'ico-info_content'})[0].text
        
    @property
    def time(self):
        return self.html_content.find_all('span', attrs = {'class': 'ico-info_content'})[1].text
    
    @property
    def author(self):
        return self.html_content.find_all('p', attrs = {'class': 'user_name'})[1].text

In [8]:
meal = Uwielbiam_dish_page('https://uwielbiam.pl/przepis/ciasteczka-thumbprint-z-nadzieniem-krowkowym-i-orzechami')
meal.link

'https://uwielbiam.pl/przepis/ciasteczka-thumbprint-z-nadzieniem-krowkowym-i-orzechami'

In [9]:
meal.title

'Ciasteczka thumbprint z nadzieniem krówkowym i orzechami'

In [10]:
meal.description

'Przedstawiamy naszą krówkowo-orzechową ciasteczkową wariację. Są to pyszne, chrupiące i bardzo efektowne ciasteczka z wyraźną nutą kakaową. Idealnie sprawdzą się jako dodatek do kawy lub kubka herbaty podczas długich jesienno-zimowych wieczorów.'

In [11]:
meal.ingredients

[['SKŁADNIKI NA CIASTO'],
 [130, 'Gramów', 'mąka pszenna tortowa Lubella'],
 [30, 'Gramów', 'Kakao DecoMorreno'],
 [110, 'Gramów', 'masło'],
 [150, 'Gramów', 'cukier'],
 [1, 'Szczypta', 'sól'],
 [2, 'Sztuki', 'jajko'],
 [1, 'Szklanka', 'Orzechy laskowe mielone'],
 ['SKŁADNIKI NA NADZIENIE'],
 [150, 'Gramów', 'Cukierki krówki'],
 [3, 'Łyżki', 'śmietanka kremowa 30%'],
 [50, 'Gramów', 'Mleczna czekolada']]

In [12]:
meal.preparation

['Wymieszaj w misce mąkę i kakao. W drugiej misce zmiksuj masło z cukrem i solą na puszystą masę. Dodaj żółtka z jajek. W następnej kolejności dodaj suche składniki i miksuj całość aż do połączenia.',
 'Przykryj ciasto folią i wstaw do lodówki na godzinę. Następnie roztrzep białko widelcem. Formuj małe kulki z ciasta, mocz je w białku i obtaczaj w orzechach.',
 'Przygotowane w ten sposób ciastka układaj na blasze w odstępach. Na środku każdego ciasteczka zrób wgłębienie. Piecz je w  nagrzanym do 180°C piekarniku przez około 10-12 minut.',
 'Upieczone ciastka ostudź, po czym ponownie zrób w nich wgłębienia. W małym rondlu rozpuść krówki ze śmietanką. Tak przygotowaną masą wypełnij wgłębienia w ciastkach.',
 'Gdy masa krówkowa zastygnie, rozpuść w kąpieli wodnej czekoladę i polej nią ciastka.']

In [13]:
meal.tags

['sól',
 'na słodko',
 'cukier',
 'masło',
 'wypieki',
 'impreza',
 'deser',
 'kakao decomorreno',
 'jajko',
 'podwieczorek',
 'kakao',
 'przystawki i przekąski',
 'mąka pszenna tortowa lubella',
 'bezmięsna',
 'śmietanka kremowa 30%',
 'urodziny imieniny',
 'grill i piknik',
 'mleczna czekolada',
 'orzechy laskowe mielone',
 'cukierki krówki']

In [14]:
meal.allergens

['Gluten', 'Laktoza', 'Mleko', 'Jajka']

In [15]:
meal.skill

'Średni'

In [16]:
meal.time

'Do 1,5 godz'

In [17]:
meal.author

'decomorreno'

### DATA EXTRACTION

In [18]:
def dish_information(link):
    """Return list with information about the meal."""
    meal = Uwielbiam_dish_page(link)
    return [meal.title, meal.description, meal.ingredients, meal.preparation, meal.tags,
           meal.allergens, meal.skill, meal.time, meal.author, meal.link]

In [19]:
def extract_data(main_link):
    """
    Function which takes as a input list with dishes links and return
    a DataFrame with dish: title, description, ingredients, preparation procedure, 
    tags and allergens.
    """
    
    # build object of the webpage
    uwielbiam = Webpage(main_link)
    
    # get list with meal links from page: 1
    # (change to: uwielbiam.all_links if you want to get all data)  
    links = uwielbiam.one_page_links(1)  
    
    # extract meal information from each page
    cookbook = {}
    for counter in tqdm(range(len(links))):
        cookbook[counter] = dish_information(links[counter])

    column_names = ["title", "description", "ingredients", "preparation", "tags", "allergens", "skill", "time", "author", "link"]
    return pd.DataFrame.from_dict(cookbook, orient='index', columns=column_names)

In [20]:
DATA = extract_data(uwielbiam.link)
DATA

100%|██████████| 20/20 [00:36<00:00,  1.84s/it]


,title,description,ingredients,preparation,tags,allergens,skill,time,author,link
0,Sałatka z brukselką i ogórkami kiszonymi Krakus,.,"[[200, Gramów, Brukselka], [0.5 Szklanki Kasza...","[Ugotować kaszę pęczak., Brukselkę cienko posi...","[dla rodziny, sól, warzywa, kolacja, oliwa z o...",None,Łatwy,Do 40 min,Krakus,http://uwielbiam.pl//przepis/salatka-z-bruksel...
1,Sałatka z gruszką i serem pleśniowym,"Chrupiąca sałatka z gruszką, serem pleśniowym ...","[[1, Szklanka, Lubella Mlekołaki Miodo Kółka],...","[Ser pleśniowy kroimy w małą kostkę., Rukolę i...","[sól, pieprz, podwieczorek, sok z cytryny, oli...",None,Łatwy,Do 15 min,lubella,http://uwielbiam.pl//przepis/salatka-z-gruszka...
2,Sałatka z winogronem i serem pleśniowym,Sałatka makaronowa z winogronami i serem pleśn...,"[[300, Gramów, makaron uszka Lubella], [20, Sz...",[Makaron gotujemy według zaleceń producenta na...,"[dla rodziny, sól, pieprz, owoce, kolacja, pod...",None,Łatwy,Do 15 min,lubella,http://uwielbiam.pl//przepis/salatka-z-winogro...
3,Sałatka z figami i szynką parmeńską,"Szybko, smacznie i baaardzo ciekawie. Jeżeli l...","[[400, Gramów, makaron Lubella Pełne Ziarno św...",[Makaron gotujemy według zaleceń producenta na...,"[sól, pieprz, kolacja, podwieczorek, sok z cyt...",None,Łatwy,Do 15 min,lubella,http://uwielbiam.pl//przepis/salatka-z-figami-...
4,Sałatka z winogronem i kurczakiem,"Pomysł na zdrową, niebanalną kolację, w słodko...","[[0.5 Sztuki Rukola], [10, Sztuk, Winogrona], ...",[Pierś kurczaka myjemy i osuszamy. Kroimy w ko...,"[warzywa, sól, pieprz, owoce, kolacja, podwiec...",None,Łatwy,Do 40 min,Łowicz,http://uwielbiam.pl//przepis/salatka-z-winogro...
5,"Sałatka ziemniaczana z bobem, boczkiem i korni...",Bób - korzystajcie póki jest dostępny świeży ;...,"[[200, Gramów, Bób], [250, Gramów, ziemniaki],...","[Warzywa ugotować, aby były al dente., Boczek ...","[obiad, dla rodziny, warzywa, sól, pieprz, kol...",None,Łatwy,Do 40 min,Krakus,http://uwielbiam.pl//przepis/salatka-ziemniacz...
6,Sałatka z pieczonych buraków i mozzarelli z dr...,"Zdrowa, lekka i przede wszystkim letnia sałatk...","[[2, Sztuki, Buraki], [2, Szklanki, Rukola], [...",[Buraki (nie obierać!) owinąć w folię i piec w...,"[warzywa, kolacja, sałatka, oliwa z oliwek, po...",None,Łatwy,"Do 1,5 godz",Łowicz,http://uwielbiam.pl//przepis/salatka-z-pieczon...
7,"Mini makaron czekoladowy z truskawkami, arbuze...",Sałatka z makaronem i owocami to deser nie tyl...,"[[130, Gramów, Makaron Lubella Mini świderki z...",[Makaron ugotować według wskazówek na opakowan...,"[polska, impreza, deser, owoce, podwieczorek, ...",None,Łatwy,Do 40 min,lubella,http://uwielbiam.pl//przepis/mini-makaron-czek...
8,"Sałatka z makaronem czekoladowym, fetą, arbuze...","Ta sałatka dowodzi, że makaron czekoladowy świ...","[[130, Gramów, Makaron Lubella Mini świderki z...",[Makaron ugotować według wskazówek na opakowan...,"[warzywa, kolacja, oliwa z oliwek, podwieczore...",None,Łatwy,Do 40 min,lubella,http://uwielbiam.pl//przepis/salatka-z-makaron...
9,Wiosenna sałatka z kalarepy z ogórkami konserw...,Warzywna sałatka z kalarepą i ogórkami konserw...,"[[1, Sztuka, Kalarepa], [3, Sztuki, ogórki kon...","[Kalarepę zetrzeć na tarce o dużych oczkach., ...","[polska, warzywa, sól, pieprz, kolacja, majone...",None,Łatwy,Do 40 min,Krakus,http://uwielbiam.pl//przepis/wiosenna-salatka-...
